In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

In [2]:
stats = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference per game stats 2012-19.csv')
stats = stats[stats['Year'] >= 2016]
print(stats.shape)
stats.head()

(2545, 30)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFT%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
2386,Quincy Acy,PF,25,SAC,59,29,14.8,2.0,3.6,0.556,0.3,0.8,0.388,1.7,2.8,0.606,0.600,0.8,1.2,0.735,1.1,2.1,3.2,0.5,0.5,0.4,0.5,1.7,5.2,2016
2387,Jordan Adams,SG,21,MEM,2,0,7.5,1.0,3.0,0.333,0.0,0.5,0.000,1.0,2.5,0.400,0.333,1.5,2.5,0.600,0.0,1.0,1.0,1.5,1.5,0.0,1.0,1.0,3.5,2016
2388,Steven Adams,C,22,OKC,80,80,25.2,3.3,5.3,0.613,0.0,0.0,NaN,3.3,5.3,0.613,0.613,1.4,2.5,0.582,2.7,3.9,6.7,0.8,0.5,1.1,1.1,2.8,8.0,2016
2389,Arron Afflalo,SG,30,NYK,71,57,33.4,5.0,11.3,0.443,1.3,3.4,0.382,3.7,7.9,0.469,0.500,1.5,1.8,0.840,0.3,3.4,3.7,2.0,0.4,0.1,1.2,2.0,12.8,2016
2390,Alexis Ajinça,C,27,NOP,59,17,14.6,2.5,5.3,0.476,0.0,0.0,0.000,2.5,5.3,0.478,0.476,0.9,1.1,0.839,1.3,3.3,4.6,0.5,0.3,0.6,0.9,2.3,6.0,2016


# Data Cleaning 

Due to trades, some players have multiple entries. The 'Tm' features highlights which team the player has played for. TOT indicates the combined stats in one season while the other entries are stats for the respective teams they played in. We will consider the stats of the entire season. We will also be considering only players who have played at least 15 games.

In [3]:
#Replacing TOT with the last team played
for y in [2016, 2017, 2018, 2019]:
    dup = []
    for i, k in enumerate(stats[stats['Year'] == y]['Player']):
        dup += [k] if stats.loc[stats['Year'] == y, 'Tm'].reset_index(drop = True)[i] == 'TOT' else []

    for j in dup:
        index = list(stats.loc[(stats['Player'] == j) & (stats['Year'] == y), 'Tm'].index)
        stats.loc[index[0], 'Tm'] = stats.loc[index[-1], 'Tm']
        
stats.drop_duplicates(['Player', 'Year'], keep = 'first', inplace = True)

In [4]:
#Dropping players who played less than 15 games
stats.reset_index(drop = True, inplace = True)

In [5]:
!pip install unidecode

You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import unidecode
for j in stats['Player'].unique():
    stats.replace(j, unidecode.unidecode(j), inplace = True)

In [7]:
stats['Player'] = stats['Player'].str.replace('.', '')

In [8]:
stats.shape

(2032, 30)

# Missing Data:

In [9]:
stats.isnull().sum()[stats.isnull().sum() > 0]

FG%       9
3P%     164
2P%      28
eFT%      9
FT%      95
dtype: int64

In [10]:
stats[['3P', '3PA', '3P%']][stats['3P%'].isnull()].describe()

,3P,3PA,3P%
count,164.0,164.0,0.0
mean,0.0,0.0,NaN
std,0.0,0.0,NaN
min,0.0,0.0,NaN
25%,0.0,0.0,NaN
50%,0.0,0.0,NaN
75%,0.0,0.0,NaN
max,0.0,0.0,NaN


In [11]:
stats[['2P', '2PA', '2P%']][stats['2P%'].isnull()].describe()

,2P,2PA,2P%
count,28.0,28.0,0.0
mean,0.0,0.0,NaN
std,0.0,0.0,NaN
min,0.0,0.0,NaN
25%,0.0,0.0,NaN
50%,0.0,0.0,NaN
75%,0.0,0.0,NaN
max,0.0,0.0,NaN


In [12]:
stats[['FT', 'FTA', 'FT%']][stats['FT%'].isnull()].describe()

,FT,FTA,FT%
count,95.0,95.0,0.0
mean,0.0,0.0,NaN
std,0.0,0.0,NaN
min,0.0,0.0,NaN
25%,0.0,0.0,NaN
50%,0.0,0.0,NaN
75%,0.0,0.0,NaN
max,0.0,0.0,NaN


NAN because is all 0

# NBA All Stars

In [13]:
allstar = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference All-Stars.csv')
allstar.head()

,Player,Year
0,Isiah Thomas,1986
1,Larry Bird,1986
2,Moses Malone,1986
3,Sidney Moncrief,1986
4,Julius Erving,1986


In [14]:
allstar['All Star'] = 1
allstar['Total All Star'] = 1

In [15]:
stats = stats.merge(allstar, on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i, 'Total All Star'] = stats.loc[stats['Player'] == i, 'Total All Star'].cumsum()
stats = stats[stats['Year'] >= 2016]

In [16]:
print(stats.shape)
stats.head()

(2037, 32)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFT%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,All Star,Total All Star
2816,Kobe Bryant,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016,1.0,18.0
0,Quincy Acy,PF,25.0,SAC,59.0,29.0,14.8,2.0,3.6,0.556,0.3,0.8,0.388,1.7,2.8,0.606,0.600,0.8,1.2,0.735,1.1,2.1,3.2,0.5,0.5,0.4,0.5,1.7,5.2,2016,0.0,0.0
240,DeAndre Jordan,C,27.0,LAC,77.0,77.0,33.7,4.6,6.6,0.703,0.0,0.0,0.000,4.6,6.6,0.704,0.703,3.5,8.0,0.430,3.5,10.3,13.8,1.2,0.7,2.3,1.4,2.7,12.7,2016,0.0,0.0
325,Raul Neto,PG,23.0,UTA,81.0,53.0,18.5,2.2,5.2,0.431,0.8,2.0,0.395,1.4,3.2,0.453,0.507,0.6,0.9,0.743,0.2,1.3,1.5,2.1,0.8,0.0,1.3,1.5,5.9,2016,0.0,0.0
324,Jameer Nelson,PG,33.0,DEN,39.0,15.0,26.6,2.9,7.9,0.368,1.1,3.7,0.299,1.8,4.2,0.429,0.438,0.8,0.9,0.857,0.5,2.5,2.9,4.9,0.6,0.1,1.7,2.6,7.7,2016,0.0,1.0


# NBA Awards

In [17]:
awards = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference Awards.csv')
awards.head()

,Unnamed: 0,Year,Player,MVP,Rookie of Year,DPOY,6th man of Year,Most Improved,Finals MVP
0,141,1953,Monk Meineke,NaN,1.0,NaN,NaN,NaN,NaN
1,140,1954,Ray Felix,NaN,1.0,NaN,NaN,NaN,NaN
2,139,1955,Bob Pettit,NaN,1.0,NaN,NaN,NaN,NaN
3,63,1956,Bob Pettit,1.0,NaN,NaN,NaN,NaN,NaN
4,138,1956,Maurice Stokes,NaN,1.0,NaN,NaN,NaN,NaN


In [18]:
awards.fillna(0, inplace = True)

In [19]:
awards['Total MVP'] = awards['MVP']
awards['Total Rookie of Year'] = awards['Rookie of Year']
awards['Total DPOY'] = awards['DPOY']
awards['Total 6th man of Year'] = awards['6th man of Year']
awards['Total Most Improved'] = awards['Most Improved']
awards['Total Finals MVP'] = awards['Finals MVP'] 

In [20]:
stats = stats.merge(awards.drop('Unnamed: 0', axis = 1), on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i , 'Total MVP'] = stats.loc[stats['Player'] == i, 'Total MVP'].cumsum()
    stats.loc[stats['Player'] == i , 'Total Rookie of Year'] = stats.loc[stats['Player'] == i, 'Total Rookie of Year'].cumsum()
    stats.loc[stats['Player'] == i , 'Total DPOY'] = stats.loc[stats['Player'] == i, 'Total DPOY'].cumsum()
    stats.loc[stats['Player'] == i , 'Total 6th man of Year'] = stats.loc[stats['Player'] == i, 'Total 6th man of Year'].cumsum()
    stats.loc[stats['Player'] == i , 'Total Most Improved'] = stats.loc[stats['Player'] == i, 'Total Most Improved'].cumsum()
    stats.loc[stats['Player'] == i , 'Total Finals MVP'] = stats.loc[stats['Player'] == i, 'Total Finals MVP'].cumsum()
stats = stats[stats['Year'] >= 2016]

# All NBA Teams

In [21]:
allnba = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference All NBA Teams.csv')
allnba.head()

,Unnamed: 0,Year,Team,Player
0,0,2019,1st,Nikola Jokić
1,1,2019,1st,Giannis Antetokounmpo
2,2,2019,1st,Paul George
3,3,2019,1st,James Harden
4,4,2019,1st,Stephen Curry


In [22]:
allrookie = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference All NBA Rookie Teams.csv')
allrookie.head()

,Unnamed: 0,Year,Team,Player
0,0,2019,1st,Luka Dončić
1,1,2019,1st,Trae Young
2,2,2019,1st,Deandre Ayton
3,3,2019,1st,Jaren Jackson
4,4,2019,1st,Marvin Bagley


In [23]:
alldefensive = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference All NBA Defensive Teams.csv')
alldefensive.head()

,Unnamed: 0,Year,Team,Player
0,0,2019,1st,Rudy Gobert
1,1,2019,1st,Paul George
2,2,2019,1st,Giannis Antetokounmpo
3,3,2019,1st,Marcus Smart
4,4,2019,1st,Eric Bledsoe


In [24]:
allnba.loc[allnba['Team'] == '1st', 'All NBA 1st Team'] = 1
allnba.loc[allnba['Team'] == '2nd', 'All NBA 2nd Team'] = 1
try:
    allnba.loc[allnba['Team'] == '3rd', 'All NBA 3rd Team'] = 1
except:
    pass

In [25]:
allrookie.loc[allrookie['Team'] == '1st', 'All Rookie 1st Team'] = 1
allrookie.loc[allrookie['Team'] == '2nd', 'All Rookie 2nd Team'] = 1

In [26]:
alldefensive.loc[alldefensive['Team'] == '1st', 'All Defensive 1st Team'] = 1
alldefensive.loc[alldefensive['Team'] == '2nd', 'All Defensive 2nd Team'] = 1

In [27]:
allnba['Total All NBA 1st Team'] = allnba['All NBA 1st Team']
allnba['Total All NBA 2nd Team'] = allnba['All NBA 2nd Team']
try:
    allnba['Total All NBA 3rd Team'] = allnba['All NBA 3rd Team']
except:
    pass

allrookie['Total All Rookie 1st Team'] = allrookie['All Rookie 1st Team']
allrookie['Total All Rookie 2nd Team'] = allrookie['All Rookie 2nd Team']

alldefensive['Total All Defensive 1st Team'] = alldefensive['All Defensive 1st Team']
alldefensive['Total All Defensive 2nd Team'] = alldefensive['All Defensive 2nd Team']

In [28]:
allnba.fillna(0, inplace = True)
allrookie.fillna(0, inplace = True)
alldefensive.fillna(0, inplace = True)

In [29]:
stats = stats.merge(allnba.drop(['Unnamed: 0', 'Team'], axis = 1), on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i , 'Total All NBA 1st Team'] = stats.loc[stats['Player'] == i, 'Total All NBA 1st Team'].cumsum()
    stats.loc[stats['Player'] == i , 'Total All NBA 2nd Team'] = stats.loc[stats['Player'] == i, 'Total All NBA 2nd Team'].cumsum()
    try:
        stats.loc[stats['Player'] == i , 'Total All NBA 3rd Team'] = stats.loc[stats['Player'] == i, 'Total All NBA 3rd Team'].cumsum()
    except:
        pass
stats = stats[stats['Year'] >= 2016]

In [30]:
stats = stats.merge(allrookie.drop(['Unnamed: 0', 'Team'], axis = 1), on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i , 'Total All Rookie 1st Team'] = stats.loc[stats['Player'] == i, 'Total All Rookie 1st Team'].cumsum()
    stats.loc[stats['Player'] == i , 'Total All Rookie 2nd Team'] = stats.loc[stats['Player'] == i, 'Total All Rookie 2nd Team'].cumsum()
stats = stats[stats['Year'] >= 2016]

In [31]:
stats = stats.merge(alldefensive.drop(['Unnamed: 0', 'Team'], axis = 1), on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i , 'Total All Defensive 1st Team'] = stats.loc[stats['Player'] == i, 'Total All Defensive 1st Team'].cumsum()
    stats.loc[stats['Player'] == i , 'Total All Defensive 2nd Team'] = stats.loc[stats['Player'] == i, 'Total All Defensive 2nd Team'].cumsum()
stats = stats[stats['Year'] >= 2016]

In [32]:
stats.shape

(2046, 58)

# Free Agency

In [33]:
free_agency = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference Free Agency.csv')
print(free_agency.shape)
free_agency.head()

(581, 5)


,Unnamed: 0,Player,New Team,Terms,Year
0,0,Kevin Durant,GSW,Signed 2-yr/$54M deal with Warriors,2016
1,1,LeBron James,CLE,Signed 3-yr/$100M deal with Cavaliers,2016
2,2,Hassan Whiteside,MIA,Signed 4-yr/$98M deal with Heat,2016
3,3,DeMar DeRozan,TOR,Signed 5-yr/$139M deal with Raptors,2016
4,4,Al Horford,BOS,Signed 4-yr/$113M deal with Celtics,2016


In [34]:
for j in free_agency['Player'].unique():
    free_agency.replace(j, unidecode.unidecode(j), inplace = True)

In [35]:
free_agency['Player'] = free_agency['Player'].str.replace('.', '')

In [36]:
salary_2017_2018 = pd.read_csv('Web-Scrapping Scripts/Hoopshype.com NBA Salaries 2016-2018.csv')
salary_2017_2018.rename({'Actual_Salary' : 'Salary'}, axis = 1, inplace = True)
salary_2017_2018.drop('Adjusted_Salary', axis = 1, inplace = True)
salary_2017_2018.head()

,Unnamed: 0,Player,Salary,Year
0,0,LeBron James,30963450,2016
1,1,Al Horford,26540100,2016
2,2,Kevin Durant,26540100,2016
3,3,James Harden,26540100,2016
4,4,Mike Conley,26540100,2016


In [37]:
salary_2019 = pd.read_csv('Web-Scrapping Scripts/Hoopshype.com NBA Salaries 2019.csv')
salary_2019.head()

,Unnamed: 0,Player,Salary,Year
0,0,Stephen Curry,40231758,2019
1,1,Russell Westbrook,38506482,2019
2,2,Chris Paul,38506482,2019
3,3,James Harden,38199000,2019
4,4,John Wall,38199000,2019


In [38]:
salary = salary_2017_2018.append(salary_2019)
salary.head()

,Unnamed: 0,Player,Salary,Year
0,0,LeBron James,30963450,2016
1,1,Al Horford,26540100,2016
2,2,Kevin Durant,26540100,2016
3,3,James Harden,26540100,2016
4,4,Mike Conley,26540100,2016


In [39]:
inconsistent_in_stats = []
for i, k in enumerate(free_agency.merge(salary, how = 'left', on = 'Player')['Player']):
    inconsistent_in_stats += [k] if free_agency.merge(salary, how = 'left', on = 'Player')['Salary'].isnull()[i] else []
    
inconsistent_in_stats

['Maurice Harkless',
 'Nene Hilario',
 'Chase Budinger',
 'Rasual Butler',
 'Ish Smith',
 'PJ Hairston',
 'Robert Sacre',
 'Cory Jefferson',
 'Lou Amundson',
 'Chris Copeland',
 'Cleanthony Early',
 'Patty Mills',
 'Tim Hardaway',
 'Nene Hilario',
 'Glenn Robinson',
 'Gary Payton',
 'Ish Smith',
 'Nene Hilario',
 'JJ Barea',
 'Glenn Robinson',
 'Cameron Reynolds',
 'Timothe Luwawu-Cabarrot',
 'Ray Spalding',
 'Walt Lemon',
 'Amida Brimah',
 'Abdul Gaddy',
 'Kaiser Gates',
 'Terry Larrier',
 'Devyn Marble',
 'Elijah Millsap']

In [40]:
salary.replace({'Moe Harkless' : 'Maurice Harkless', 'Nenê' : 'Nene Hilario', 'Ishmael Smith' : 'Ish Smith', 'Patrick Mills' : 'Patty Mills',
                'Tim Hardaway Jr' : 'Tim Hardaway', 'Glenn Robinson III' : 'Glenn Robinson', 'Gary Payton II' : 'Gary Payton',
                'Jose Juan Barea' : 'JJ Barea', 'Timothe Luwawu' : 'Timothe Luwawu-Cabarrot'}, inplace = True)

In [41]:
free_agency1 = free_agency.merge(salary[['Player', 'Salary', 'Year']], how = 'inner', on = ['Player', 'Year'])
print(free_agency1.shape)
free_agency1.head()

(513, 6)


,Unnamed: 0,Player,New Team,Terms,Year,Salary
0,0,Kevin Durant,GSW,Signed 2-yr/$54M deal with Warriors,2016,26540100
1,1,LeBron James,CLE,Signed 3-yr/$100M deal with Cavaliers,2016,30963450
2,2,Hassan Whiteside,MIA,Signed 4-yr/$98M deal with Heat,2016,22116750
3,3,DeMar DeRozan,TOR,Signed 5-yr/$139M deal with Raptors,2016,26540100
4,4,Al Horford,BOS,Signed 4-yr/$113M deal with Celtics,2016,26540100


In [42]:
free_agency1.replace({'Nene Hilario' : 'Nene', 'Tim Hardaway' : 'Tim Hardaway Jr', 'Glenn Robinson' : 'Glenn Robinson III',
                      'Derrick Jones' : 'Derrick Jones Jr', 'Kelly Oubre' : 'Kelly Oubre Jr', 'Frank Mason' : 'Frank Mason III'},
                      inplace = True)

In [43]:
NBA_Data = stats.merge(free_agency1[['Player', 'Salary', 'Year']], how = 'inner')

In [44]:
print(NBA_Data.shape)
NBA_Data.head()

(499, 59)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFT%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,All Star,Total All Star,MVP,Rookie of Year,DPOY,6th man of Year,Most Improved,Finals MVP,Total MVP,Total Rookie of Year,Total DPOY,Total 6th man of Year,Total Most Improved,Total Finals MVP,All NBA 1st Team,All NBA 2nd Team,All NBA 3rd Team,Total All NBA 1st Team,Total All NBA 2nd Team,Total All NBA 3rd Team,All Rookie 1st Team,All Rookie 2nd Team,Total All Rookie 1st Team,Total All Rookie 2nd Team,All Defensive 1st Team,All Defensive 2nd Team,Total All Defensive 1st Team,Total All Defensive 2nd Team,Salary
0,Steve Novak,PF,32.0,MIL,10.0,0.0,4.4,0.8,1.8,0.444,0.7,1.5,0.467,0.1,0.3,0.333,0.639,0.1,0.1,1.000,0.0,0.5,0.5,0.0,0.0,0.0,0.1,0.3,2.4,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1551659
1,Ish Smith,PG,27.0,PHI,77.0,53.0,29.1,5.2,12.6,0.411,0.7,2.2,0.329,4.4,10.4,0.428,0.440,1.6,2.3,0.693,0.6,3.4,4.0,6.5,1.1,0.3,2.3,1.7,12.6,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6000000
2,JR Smith,SG,30.0,CLE,77.0,77.0,30.7,4.6,11.0,0.415,2.6,6.6,0.400,1.9,4.4,0.438,0.535,0.6,0.9,0.634,0.6,2.3,2.8,1.7,1.1,0.3,0.8,2.6,12.4,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12800000
3,Andre Drummond,C,22.0,DET,81.0,81.0,32.9,6.8,13.1,0.521,0.0,0.1,0.333,6.8,13.0,0.522,0.522,2.6,7.2,0.355,4.9,9.9,14.8,0.8,1.5,1.4,1.9,3.0,16.2,2016,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,22116750
4,Trevor Booker,PF,28.0,UTA,79.0,2.0,20.7,2.4,4.9,0.490,0.2,0.5,0.293,2.3,4.4,0.513,0.505,0.9,1.3,0.670,2.1,3.6,5.7,1.1,0.7,0.5,1.0,2.1,5.9,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9250000


In [45]:
NBA_Data['Year'].value_counts()

2019    148
2016    135
2018    109
2017    107
Name: Year, dtype: int64

In [47]:
NBA_Data.to_csv('Free Agency Data 2016-2019', index = False)